In [1]:
import cv2
from pathlib import Path
import shutil

def crop_center_top(frame, crop_width, crop_height):
    """
    Crop the frame to the center horizontally and slightly towards the top vertically.
    """
    h, w = frame.shape[:2]
    x1 = int((w - crop_width) / 2)
    y1 = int((h - crop_height) * 0.35)  # 35% arriba, 65% abajo
    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(x1 + crop_width, w)
    y2 = min(y1 + crop_height, h)
    return frame[y1:y2, x1:x2]

def process_avi_video(input_path, output_path, crop_fraction=0.6):
    """
    Read AVI video, crop each frame, and save as a new AVI file.
    crop_fraction: Size of the crop relative to original (0 < crop_fraction <= 1)
    """
    cap = cv2.VideoCapture(str(input_path))
    if not cap.isOpened():
        print(f"Failed to open {input_path}")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    crop_width = int(width * crop_fraction)
    crop_height = int(height * crop_fraction)

    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (crop_width, crop_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cropped = crop_center_top(frame, crop_width, crop_height)
        out.write(cropped)

    cap.release()
    out.release()

def process_dataset(raw_dataset_path, output_dataset_path, crop_fraction=0.6, overwrite=False):
    """
    Process all AVI videos in raw_dataset_path, cropping and saving to output_dataset_path.
    """
    raw_root = Path(raw_dataset_path)
    output_root = Path(output_dataset_path)

    if output_root.exists() and overwrite:
        shutil.rmtree(output_root)
    output_root.mkdir(parents=True, exist_ok=True)

    for class_folder in raw_root.iterdir():
        if not class_folder.is_dir():
            continue
        output_class = output_root / class_folder.name
        output_class.mkdir(exist_ok=True)
        for video_file in class_folder.glob("*.avi"):
            output_video = output_class / video_file.name
            print(f"Processing {video_file} -> {output_video}")
            process_avi_video(video_file, output_video, crop_fraction=crop_fraction)

if __name__ == "__main__":
    # CONFIGURA TUS PARÁMETROS AQUÍ
    RAW_DATASET_PATH = "raw_dataset"
    OUTPUT_DATASET_PATH = "cropped_dataset"
    CROP_FRACTION = 0.6   # 0.6 significa recortar al 60% del tamaño original (zoom-in)
    OVERWRITE = True

    process_dataset(
        RAW_DATASET_PATH,
        OUTPUT_DATASET_PATH,
        crop_fraction=CROP_FRACTION,
        overwrite=OVERWRITE
    )

Processing raw_dataset\ABUELO\0016_0000_0000.avi -> cropped_dataset\ABUELO\0016_0000_0000.avi
Processing raw_dataset\ABUELO\0016_0000_0001.avi -> cropped_dataset\ABUELO\0016_0000_0001.avi
Processing raw_dataset\ABUELO\0016_0000_0002.avi -> cropped_dataset\ABUELO\0016_0000_0002.avi
Processing raw_dataset\ABUELO\0016_0000_0003.avi -> cropped_dataset\ABUELO\0016_0000_0003.avi
Processing raw_dataset\ABUELO\0016_0001_0000.avi -> cropped_dataset\ABUELO\0016_0001_0000.avi
Processing raw_dataset\ABUELO\0016_0001_0001.avi -> cropped_dataset\ABUELO\0016_0001_0001.avi
Processing raw_dataset\ABUELO\0016_0001_0002.avi -> cropped_dataset\ABUELO\0016_0001_0002.avi
Processing raw_dataset\ABUELO\0016_0001_0003.avi -> cropped_dataset\ABUELO\0016_0001_0003.avi
Processing raw_dataset\ABUELO\0016_0002_0000.avi -> cropped_dataset\ABUELO\0016_0002_0000.avi
Processing raw_dataset\ABUELO\0016_0002_0001.avi -> cropped_dataset\ABUELO\0016_0002_0001.avi
Processing raw_dataset\ABUELO\0016_0002_0002.avi -> cropped_